In [175]:
import pandas as pd
import scipy.stats
import statsmodels.formula.api as sm

In [114]:
# Read in our files.
train_features = pd.read_csv('train_features_2013-03-07.csv')
train_salaries = pd.read_csv('train_salaries_2013-03-07.csv')
test_data = pd.read_csv('test_features_2013-03-07.csv')

In [115]:
# Join train_{features, salaries} on jobId as primary key.
train_data = pd.merge(train_features, train_salaries, on='jobId', how='inner')

In [116]:
# In-place drop rows with any NaN value.
train_data.dropna(how='any', inplace=True)
test_data.dropna(how='any', inplace=True)

In [79]:
# Drop entries where salary = 0. Keep these in for now.
# train_data = train_data[train_data.salary != 0]

In [117]:
# Dictionary for mapping variable names to integers.
mapping = {}

In [118]:
# Assign integers for these (string) variables.
names = ['companyId', 'industry', 'major', 'jobType', 'degree']
for name in names:
    for i in enumerate(train_data[name].unique()):
        if name not in mapping.keys():
            mapping[name] = {i[1]: i[0]}
        else:
            mapping[name].update({i[1]: i[0]})
    train_data[name] = train_data[name].replace(mapping[name])
    test_data[name] = test_data[name].replace(mapping[name])

In [119]:
# Look at the Pearson correlation coefficient for each of the 7 non-salary variables with salary.
names = ['companyId', 'industry', 'major', 'jobType', 'degree', 'yearsExperience', 'milesFromMetropolis']
for name in names:
    x = scipy.stats.pearsonr(train_data.salary, train_data[name])
    print(name, x[0])

companyId 0.00122280692462
industry -0.0465353484859
major 0.274073629907
jobType -0.311428572442
degree -0.153723354454
yearsExperience 0.375012699594
milesFromMetropolis -0.297666352756


In [191]:
# Create our linear model with the 5 variables with highest absolute value for correlation coefficient.
olsmod = sm.ols(formula="salary ~ yearsExperience + major + degree + jobType + milesFromMetropolis", data=train_data)

In [192]:
# Fit the model.
olsres = olsmod.fit()

In [193]:
# Summary of results.
print(olsres.summary())

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                       0.402
Model:                            OLS   Adj. R-squared:                  0.402
Method:                 Least Squares   F-statistic:                 1.345e+05
Date:                Fri, 01 Jul 2016   Prob (F-statistic):               0.00
Time:                        14:13:46   Log-Likelihood:            -4.8181e+06
No. Observations:             1000000   AIC:                         9.636e+06
Df Residuals:                  999994   BIC:                         9.636e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
Intercept             124.8728    

In [194]:
# This gives us the actual coefficients for the linear model.
olsres.params

Intercept              124.872827
yearsExperience          2.014110
major                    3.848141
degree                  -2.756044
jobType                 -4.968143
milesFromMetropolis     -0.399825
dtype: float64

In [195]:
# Insert a column in train_data for our predicted values.
train_data['salary_predicted'] = pd.Series(olsres.predict()).astype(int)

In [196]:
# A quick look at the mean of the difference of the predicted and given salaries.
(train_data.salary - train_data.salary_predicted).mean()

0.49969000000000002

In [197]:
# A quick look at the standard deviation of the difference of the predicted and given salaries.
(train_data.salary - train_data.salary_predicted).std()

29.938780740612881

In [198]:
# Insert a column in test_data for our predicted salary values as ints.
test_data['salary'] = pd.Series(olsres.predict()).astype(int)

In [170]:
# Save our results to file.
header = ['jobId', 'salary']
test_data.to_csv('test_salaries.csv', columns=header)